In [1]:
import torch
print("PyTorch has version {}".format(torch.__version__))

!pip install -q dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

PyTorch has version 1.12.1+cu113
     |████████████████████████████████| 239.2 MB 1.0 kB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 280 kB 10.3 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
     |████████████████████████████████| 52 kB 972 kB/s 
     |████████████████████████████████| 109 kB 62.6 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 29.1 MB 87.4 MB/s 
     |████████████████████████████████| 103 kB 39.6 MB/s 
     |████████████████████████████████| 41 kB 202 kB/s 
     |████████████████████████████████| 3.2 MB 43.2 MB/s 
     |████████████████████████████████| 500 kB 39.1 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 121 kB 65.5 MB/s 
     |████████████████████████████████| 100 kB 8.6 MB/s 
     |████████████████████████████████| 90 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 

In [2]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import time
import dgl

In [3]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
from transformers import BertTokenizer, BertLMHeadModel,BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokenized_text = [tokenizer.tokenize(sen) for sen in train_df.claim.values] 

max_lenth = 0 
for i in tokenized_text:
  if len(i) >= max_lenth:
    max_lenth = len(i)

print(max_lenth)

96


In [7]:
from keras_preprocessing.sequence import pad_sequences

print('padding')
MAX_LEN = 100
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]
print('input_ids[0]:',input_ids[0])

padding
input_ids[0]: [2106 1037 8839 2450 2695 1037 8257 2055 8398 5026 2067 8864 1029    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [8]:
attention_masks = []
 
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
 
print('attention_masks[0]:',attention_masks[0])

attention_masks[0]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [9]:
input_ids.shape

(5277, 100)

In [10]:
len(attention_masks)

5277

In [27]:
from tqdm import tqdm
sentence_vec = [] 

model = BertModel.from_pretrained('bert-base-uncased',
                                    output_hidden_states = True, 
                                    )
model.eval()

for i,j in zip(tqdm(input_ids),attention_masks):
  tokens_tensor = torch.tensor([i])
  segments_tensors = torch.tensor([j])

  
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

  
  token_vecs = hidden_states[-2][0]
  # # Calculate the average of all 36 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  sentence_vec.append(sentence_embedding)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5277/5277 [38:44<00:00,  2.27it/s]


In [11]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([input_ids[0]])
segments_tensors = torch.tensor([attention_masks[0]])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [13]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs

In [14]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[0][0][0]))


Number of layers: 3   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 100
Number of hidden units: 768


In [ ]:
hidden_states[2][0][0][0]
sentence_embedding = torch.mean(token_vecs, dim=0)

In [87]:
# `hidden_states` has shape [13 x 1 x 36 x 768]

# `token_vecs` is a tensor with shape [36 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 36 token vectors.
sentence_embedding1 = torch.mean(token_vecs, dim=0)



In [ ]:
sentence_embedding1

In [92]:
tmp = []
tmp.append(sentence_embedding)
tmp.append(sentence_embedding1)

In [93]:
tmp

[tensor(-0.0422), tensor(0.2926)]

In [49]:
##edge 연결 
import ast
keybert_list = [ast.literal_eval(i) for i in train_df.keybert_keywords]
print(len(keybert_list))

ner_list = [ast.literal_eval(i) for i in train_df.ner_keywords]
print(len(ner_list))

5277
5277


In [50]:
dic_array = np.zeros((len(train_df),len(train_df)))
print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(ner_list):
  for j, j_m in enumerate(ner_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 

print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(keybert_list):
  for j, j_m in enumerate(keybert_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 


print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
0.0
(5277, 5277)
1328495.0
(5277, 5277)
1382295.0


In [51]:
adj_t = torch.nonzero(torch.tensor(dic_array)).T
print("== adj to edge list: \n", adj_t)

== adj to edge list: 
 tensor([[   0,    0,    0,  ..., 5276, 5276, 5276],
        [   0,    5,    7,  ..., 5161, 5176, 5276]])


In [52]:
u,v = adj_t[0], adj_t[1]
g = dgl.graph((u, v))
g

Graph(num_nodes=5277, num_edges=1382295,
      ndata_schemes={}
      edata_schemes={})

In [38]:
g.ndata['features'] = torch.tensor(int).float()

In [39]:
##label 달기
labels = train_df.label.values
print(len(labels))

g.ndata['label'] = torch.tensor(labels)
g

5277


Graph(num_nodes=5277, num_edges=1382295,
      ndata_schemes={'features': Scheme(shape=(768,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [ ]:
lnt[0]

In [41]:
nids = np.random.permutation(np.arange(g.number_of_nodes()))

#test split
test_size = int(len(nids) * 0.2)
test_id = nids[:test_size]

#train, val split
train_id = nids[test_size:]
val_size = int(len(train_id) * 0.1)

train_id = train_id[val_size:]
val_id = train_id[:val_size]

print("train_size: ", len(train_id), "test_size: ", len(test_id), "val_size: ", len(val_id) )

train_size:  3800 test_size:  1055 val_size:  422


In [42]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats,num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


from dgl.nn import SAGEConv

# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats,h_feat,num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feat, 'mean')
        self.conv3 = SAGEConv(h_feat, num_classes, 'mean')
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        return h




In [43]:
# model = GCN(g.ndata['features'].shape[1], 64, 2)
model = GraphSAGE(g.ndata['features'].shape[1],128, 64,2)
model

GraphSAGE(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=768, out_features=128, bias=False)
    (fc_neigh): Linear(in_features=768, out_features=128, bias=False)
  )
  (conv2): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=128, out_features=64, bias=False)
    (fc_neigh): Linear(in_features=128, out_features=64, bias=False)
  )
  (conv3): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=64, out_features=2, bias=False)
    (fc_neigh): Linear(in_features=64, out_features=2, bias=False)
  )
  (dropout): Dropout(p=0.4, inplace=False)
)

In [44]:
def train(g,features,labels,train_mask,val_mask):
    model.train()

    optimizer =  torch.optim.Adam(model.parameters(), lr=0.01)
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    best_val_acc = 0
    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

    #####################################################################
    pred = logits.argmax(1)
    train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
    val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc

    return logits, loss, pred, train_acc, val_acc, best_val_acc

In [45]:
def test(g,features,labels,test_mask):
  with torch.no_grad():
    model.eval()

  ############# 5) Your code here ############
  # loss function : cross entropy
    logits = model(g, features)
    loss = F.cross_entropy(logits[test_mask], labels[test_mask])
  ##############################################

    pred = logits.argmax(1)
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    return logits, loss, pred, test_acc

In [46]:
def main(epoch, g, model,train_id, test_id, val_id): 

  features = g.ndata['features']
  labels = g.ndata['label']
  train_mask = train_id
  test_mask = test_id
  val_mask = val_id


  # train
  for e in range(epoch):
    logits, loss, pred, train_acc, val_acc, best_val_acc = train(g,features,labels,train_mask,val_mask)
    print('In epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f} (best {:.3f})'.format(
                e, loss, train_acc,val_acc, best_val_acc))


  # test
  logits, loss, pred, test_acc = test(g,features,labels,test_mask)
  print('[Test result] loss: {:.3f}, test acc: {:.3f}'.format(loss, test_acc))


In [47]:
g = dgl.add_self_loop(g)

In [48]:
main(50,g,model,train_id, test_id, val_id)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 4.640, train acc: 0.510, val acc: 0.505 (best 0.505)
In epoch 1, loss: 47.758, train acc: 0.491, val acc: 0.455 (best 0.455)
In epoch 2, loss: 6.080, train acc: 0.507, val acc: 0.550 (best 0.550)
In epoch 3, loss: 28.293, train acc: 0.491, val acc: 0.455 (best 0.455)
In epoch 4, loss: 4.002, train acc: 0.516, val acc: 0.521 (best 0.521)
In epoch 5, loss: 19.047, train acc: 0.491, val acc: 0.455 (best 0.455)
In epoch 6, loss: 3.842, train acc: 0.518, val acc: 0.543 (best 0.543)
In epoch 7, loss: 17.453, train acc: 0.491, val acc: 0.455 (best 0.455)
In epoch 8, loss: 2.565, train acc: 0.521, val acc: 0.491 (best 0.491)
In epoch 9, loss: 5.037, train acc: 0.493, val acc: 0.455 (best 0.455)
In epoch 10, loss: 5.861, train acc: 0.510, val acc: 0.545 (best 0.545)
In epoch 11, loss: 2.170, train acc: 0.510, val acc: 0.512 (best 0.512)
In epoch 12, loss: 3.945, train acc: 0.508, val acc: 0.538 (best 0.538)
In epoch 13, loss: 2.936, train acc: 0.494, val acc: 0.493 (best 0.493